In [1]:
import more_itertools
import itertools
from itertools import product
import numpy as np
from numpy.polynomial.polynomial import Polynomial

In [83]:
K = 2
L = K
B = 6
mpl = [1, 5]
mu=[
    [0 ] + [1*i for i in range(1, mpl[0]+1)] + [1*mpl[0] for i in range(mpl[0]+1, B+1)],
    [0 ] + [0.8*i for i in range(1, mpl[1]+1)] + [0.8*mpl[1] for i in range(mpl[1]+1, B+1)]
   ]
gamma=[3/4,1/4]
lmbd = 1.6

In [84]:
def W(V):
    Vlist = list(set(np.ndarray.flatten(V))) # flatten and remove duplicates

    s=0
    V0index=-1

    while s<lmbd:
        V0index+=1
        V0=Vlist[V0index]
        ihat=[-1 for i in range(L)]
        for l in range(L):
            for i in range(B):
                if V[l,i] <= V0+0.0000001:
                    ihat[l] = i
        s=0
        for l in range(L):
            s+=gamma[l]*mu[l][ihat[l]+1]

    
    if V0>B: # some queue types are not used -> there will be TONS of data loss
        return 100 # practical infinity
    
    if (min(V[0])>max(V[1])):
        return 100 # practical infinity

    if (min(V[1])>max(V[0])):
        return 100 # practical infinity

    icheck =[0,0]
    for l in range(L):
        i=0
        while (V[l][i]<V0):
            i+=1
            icheck[l]=i

    
    if max(icheck) >= B: # there will be data loss
        return 100
    
#    print('V0: ', V0)
#    print('icheck: ', icheck)
#    print('ihat: ', ihat)
    
    dom = [0,10000]; # technical value for Polynomial type variables; it should be the same and large enough for all Polynomials
    denomlist = [Polynomial([1],domain=dom,window=dom) for l in range(L)]
    numerlist = [Polynomial([0],domain=dom,window=dom) for l in range(L)]
    prodlist=[[1 for i in range(icheck[l],ihat[l]+2)] for l in range(L)]
    prodlist2 = [[1 for i in range(icheck[l],ihat[l]+1)] for l in range(L)]
    for l in range(L):
        for i in range(icheck[l]+1,ihat[l]+2):
            prodlist[l][i-icheck[l]] = prodlist[l][i-1-icheck[l]]*1/(mu[l][i])
        denomlist[l] = Polynomial(prodlist[l],domain=dom,window=dom)
        for i in range(icheck[l]+1,ihat[l]+1):
            prodlist2[l][i-icheck[l]] = prodlist2[l][i-1-icheck[l]]*1/(mu[l][i])
        if prodlist2[l] == []:
            numerlist[l] = Polynomial([0],domain=dom,window=dom)
        else:
            numerlist[l] = Polynomial(prodlist2[l],domain=dom,window=dom)
        if icheck[l]>ihat[l]:
            numerlist[l] = Polynomial([0],domain=dom,window=dom)

#    print('prodlist: ',prodlist)
#    print('prodlist2: ',prodlist2)
#    print('numerlist: ',numerlist)
#    print('denomlist :',denomlist)
    lmbdunumer = Polynomial([0],domain=dom,window=dom) 
    lmbdudenom = Polynomial([1],domain=dom,window=dom)
    d0numer = Polynomial([0],domain=dom,window=dom) 
    d0denom = Polynomial([1],domain=dom,window=dom)
    for l in range(L):
        if mu[l][icheck[l]]>0: # keeps the fraction simplified as much as possible
            lmbdunumer = np.polymul(lmbdunumer,denomlist[l]) + np.polymul(lmbdudenom,Polynomial([gamma[l]*mu[l][icheck[l]]],domain=dom,window=dom))
            lmbdudenom = np.polymul(lmbdudenom,denomlist[l])
        
    for l in range(L):
        d0numer = np.polymul(d0numer,denomlist[l]) + gamma[l]*np.polymul(d0denom,numerlist[l])
        d0denom = np.polymul(d0denom,denomlist[l])

#    print(lmbdunumer)
#    print(lmbdudenom)
    eqx=np.polymul(np.polymul(Polynomial([0,1],domain=dom,window=dom),d0numer),lmbdudenom)-lmbd*np.polymul(d0denom,lmbdudenom)+np.polymul(lmbdunumer,d0denom)

    xlist=eqx[0].roots()
    
#    print(xlist)
    
    x=[x for x in xlist if x>0][0].real
#    print('x: ',x)

    nucheck = [0 for l in range(L)]
    for l in range(L):
        slist=[0 for i in range(ihat[l]+2)]
        slist[icheck[l]]=1
        for i in range(icheck[l],ihat[l]+1):
            slist[i+1] = slist[i]*x/mu[l][i+1]
        nucheck[l]=gamma[l]/sum(slist)

    nu= [[0 for i in range(B+1)] for l in range(L)]
    for l in range(L):
        nu[l][icheck[l]] = nucheck[l];
        for i in range(icheck[l]+1,ihat[l]+2):
            nu[l][i] = nu[l][i-1]*x/mu[l][i]

#    print('nu: ',nu)
    
    A = sum([sum([i*nu[l][i] for i in range(B+1)]) for l in range(L)]) # average queue length

    return A/lmbd


In [85]:
types = [i for i in range(K)]
lengths = [i for i in range(B)]
alllengths = [i for i in itertools.product(types, lengths)]

alllengths_grouped = []
for i in range(K):
    alllengths_grouped.append([])
    first_batch = []
    for j in range(mpl[i]):
        first_batch.append(alllengths[i * B + j])
    alllengths_grouped[i].append(first_batch)
    for j in range(mpl[i], B):
        alllengths_grouped[i].append([alllengths[i * B + j]])

keys = []
for i in range(K):
    keys += [i for j in range(B - mpl[i] + 1)]

sorted_key_orders = np.array(sorted(more_itertools.distinct_permutations(keys)))

def is_increasing(x):   # to allow equality only at pairs sorted according to type (to avoid listing the same V multiple times)
    if x <= 0:
        return 0
    else:
        return 1
    
is_increasing_vect = np.vectorize(is_increasing)

def all_possible_weights(diff, length):
    weighting_generator = list(product([0,1], repeat=np.sum(diff)))
    all_weights = []
    for i in range(len(weighting_generator)):
        new_weighting = np.ones(length)
        c = 0
        for j in range(len(diff)):
            if diff[j] == 0:
                new_weighting[j + 1:] += 1
            else: # elif  diff[j] == 1
                if weighting_generator[i][c] == 1:
                    new_weighting[j + 1:] += 1
                # else new_weighting[j + 1:] += 0
                c += 1
        all_weights.append(new_weighting)
    return all_weights

In [86]:
all_orders = {}
for order in sorted_key_orders:
    l = len(order)
    diff_vector = is_increasing_vect(np.diff(order))
    all_orders[tuple(order)] = all_possible_weights(diff_vector, l)
    
results = []
for nums in all_orders:
    for weights in all_orders[nums]:
        new_item = []
        is_first = []
        for i in range(len(mpl)):
            new_item.append([])
            is_first.append(True)
        for w in range(len(weights)):
            if is_first[nums[w]]:
                for j in range(mpl[nums[w]]):
                    new_item[nums[w]].append(weights[w])
                is_first[nums[w]] = False
            else:
                new_item[nums[w]].append(weights[w])
        results.append(new_item)

vlist = np.array(results) # contains all possible V prioritization functions

In [87]:
Wopt=100
Vopt = 0
for V in vlist:
    Wcurr = W(V)
    if Wcurr < Wopt:
        Wopt = Wcurr
        Vopt = V

print(Vopt)
print(Wopt)

[[1. 3. 4. 5. 6. 7.]
 [2. 2. 2. 2. 2. 7.]]
1.1328125
